# GNSS time sync evaluation on VersaVIS
Zurich, 14. Jan 2020, brik@ethz.ch

In [234]:
def getVersavisError(file, clock_type):
    import rosbag_pandas
    import pandas as pd

    # Open file
    df = rosbag_pandas.bag_to_dataframe(file)
    # Time stamps to seconds.    
    df_piksi = pd.to_datetime(df['/rover/piksi/attitude_receiver_0/ros/ext_event/stamp/data/secs'], unit='s') + pd.to_timedelta(df['/rover/piksi/attitude_receiver_0/ros/ext_event/stamp/data/nsecs'], unit='ns')
    df_versavis = pd.to_datetime(df['/versavis/external_event/data/secs'], unit='s') + pd.to_timedelta(df['/versavis/external_event/data/nsecs'], unit='ns') 
    
    # To datetime.
    df_piksi = df_piksi.reset_index()
    df_piksi.columns = ['t_arrival', 't_piksi']
    df_piksi['t_arrival'] = pd.to_datetime(df_piksi['t_arrival'], unit='s')
    df_piksi.dropna(inplace=True)

    df_versavis = df_versavis.reset_index()
    df_versavis.columns = ['t_arrival', 't_versavis']
    df_versavis['t_arrival'] = pd.to_datetime(df_versavis['t_arrival'], unit='s')
    df_versavis.dropna(inplace=True)

    # Find nearest arrival time pairs.
    df_pairs = pd.merge_asof(df_versavis, df_piksi, on='t_arrival', tolerance=pd.Timedelta(milliseconds=100), direction='nearest')
    
    # Calculate error.
    df_diff = -df_pairs[['t_versavis', 't_piksi']].diff(axis=1)
    
    df_error = pd.concat([df_pairs['t_piksi'], df_diff['t_piksi']], axis=1, sort=True)
    df_error.columns = ['t', clock_type]
    df_error = df_error.set_index('t')
    
    # Reindex to start from the first recorded second.
    t0 = df_error.index[0]
    t0 = t0.replace(nanosecond=0)
    t0 = t0.replace(microsecond=0)
    df_error.index = df_error.index - t0
    
    # Convert to index to seconds and value to s.   
    import numpy as np
    df_error[clock_type] = df_error[clock_type] / np.timedelta64(1,'s')
    
    # Reindex.
    df_error.reset_index(inplace=True)
    df_error['t'] = df_error['t'] / np.timedelta64(1, 's')
       
    #import pdb; pdb.set_trace()
    return df_error


In [421]:
def plotTimeSeries(file_XOSC32K, file_DFLL48M, file_EXT10MHZ, file_ROSTIMENOW, start, width, height, fontsize):

    # Calculate difference.
    df_error_XOSC32K = getVersavisError(file_XOSC32K, 'XOSC32K').abs()
    df_error_DFLL48M = getVersavisError(file_DFLL48M, 'DFLL48M').abs()
    df_error_EXT10MHZ = getVersavisError(file_EXT10MHZ, 'EXT10MHZ').abs()
    df_error_ROSTIMENOW = getVersavisError(file_ROSTIMENOW, 'ROSTIMENOW').abs()

    # Crop time to minmax
    max_t_XOSC32K = max(df_error_XOSC32K['t'])
    max_t_DFLL48M = max(df_error_DFLL48M['t'])
    max_t_EXT10MHZ = max(df_error_EXT10MHZ['t'])
    max_t_ROSTIMENOW = max(df_error_ROSTIMENOW['t'])
    minmax = min([max_t_XOSC32K, max_t_DFLL48M, max_t_EXT10MHZ, max_t_ROSTIMENOW])

    # Plot time series.
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(width,height))
    df_error_XOSC32K.loc[df_error_XOSC32K['t'] < minmax].plot(ax=ax, x='t', fontsize=fontsize, logy=True)
    df_error_DFLL48M.loc[df_error_DFLL48M['t'] < minmax].plot(ax=ax, x='t', fontsize=fontsize, logy=True)
    df_error_EXT10MHZ.loc[df_error_EXT10MHZ['t'] < minmax].plot(ax=ax, x='t', fontsize=fontsize, logy=True)
    df_error_ROSTIMENOW.loc[df_error_ROSTIMENOW['t'] < minmax].plot(ax=ax, x='t', fontsize=fontsize, logy=True)

    ax.set_xlabel('Duration since experiment start [hh:mm:ss]', fontsize=fontsize)
    ax.set_ylabel('Absolute time error [s]', fontsize=fontsize)
    
    # Make x axis show seconds.
    #import pdb; pdb.set_trace()
    import math
    import numpy as np
    import matplotlib
    import datetime
    x = np.linspace(0, math.ceil(minmax))
    def timeTicks(x, pos):                                                                                                                                                                                                                                                         
        d = datetime.timedelta(seconds=x)                                                                                                                                                                                                                                          
        return str(d) 
    
    formatter = matplotlib.ticker.FuncFormatter(timeTicks)
    ax.xaxis.set_major_formatter(formatter)
    
    # Mark ROSTIMENOW convergence time.
    ax.axvline(x=start, color='black', linestyle='--')
    ax.yaxis.grid(True)
    
    fig.savefig("plots/timeseries.pdf", bbox_inches='tight')
    
    

In [307]:
def createErrorHistogram(df_error, clock):
    # Create non-uniform bins.  Unit in s.
    import numpy as np
    import pandas as pd
    start = -10
    stop = 0
    end = stop - start + 1
    bins = np.logspace(start, stop, end)
    # Create histogram.
    y, x = np.histogram(df_error[clock].to_numpy(), bins=bins)
    # Correct bin placement
    x = x[1:]
    # Turn into pandas Series
    s = pd.Series(y, x)
    df = s.to_frame()
    df.columns=[clock]
    return df

def plotHistogram(file_XOSC32K, file_DFLL48M, file_EXT10MHZ, file_ROSTIMENOW, start, width, height, fontsize):
    # Calculate difference.
    df_error_XOSC32K = getVersavisError(file_XOSC32K, 'XOSC32K')
    df_error_DFLL48M = getVersavisError(file_DFLL48M, 'DFLL48M')
    df_error_EXT10MHZ = getVersavisError(file_EXT10MHZ, 'EXT10MHZ')
    df_error_ROSTIMENOW = getVersavisError(file_ROSTIMENOW, 'ROSTIMENOW')
    
    # Start data when ROSTIMENOW converged.
    df_error_XOSC32K = df_error_XOSC32K[df_error_XOSC32K['t'] > start]
    df_error_DFLL48M = df_error_DFLL48M[df_error_DFLL48M['t'] > start]
    df_error_EXT10MHZ = df_error_EXT10MHZ[df_error_EXT10MHZ['t'] > start]
    df_error_ROSTIMENOW = df_error_ROSTIMENOW[df_error_ROSTIMENOW['t'] > start]
    
    # Get absolute error.
    import pandas as pd
    df_error = pd.concat([df_error_XOSC32K['XOSC32K'],df_error_DFLL48M['DFLL48M'], df_error_EXT10MHZ['EXT10MHZ'], df_error_ROSTIMENOW['ROSTIMENOW']], axis=1).dropna().abs()
    
    hist_XOSC32K = createErrorHistogram(df_error, 'XOSC32K')
    hist_DFLL48M = createErrorHistogram(df_error, 'DFLL48M')
    hist_EXT10MHZ = createErrorHistogram(df_error, 'EXT10MHZ')
    hist_ROSTIMENOW = createErrorHistogram(df_error, 'ROSTIMENOW')
    df_hist = pd.concat([hist_XOSC32K,hist_DFLL48M, hist_EXT10MHZ, hist_ROSTIMENOW], axis=1)

    ax = df_hist.plot(kind='bar', figsize=(width,height), alpha=0.5, fontsize=fontsize)

    ax.set_xlabel('Timestamp error', fontsize=fontsize)
    ax.set_xticklabels(['1 ns', '10 ns', '100 ns', '1 us', '10 us', '100 us', '1 ms', '10 ms', '100 ms', '1 s'], rotation='horizontal')
    ax.set_ylabel('Frequency', fontsize=fontsize)

    # Draw mean.
    def drawMean(df, column):
        import matplotlib.pyplot as plt

        i = df_error.columns.get_loc(column)
        x = df_error[column]

        color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
        #plt.axvline(x.mean(), linestyle='dashed', linewidth=1, color=color)
        #min_ylim, max_ylim = plt.ylim()
        #plt.text(x.mean()*1.05, max_ylim*0.9, 'Mean:\n{:.2f}us'.format(x.mean()))

    drawMean(df_error, 'XOSC32K')
    drawMean(df_error, 'DFLL48M')
    drawMean(df_error, 'EXT10MHZ')
    drawMean(df_error, 'ROSTIMENOW')

In [418]:
def plotViolin(file_XOSC32K, file_DFLL48M, file_EXT10MHZ, file_ROSTIMENOW, start, width, height, fontsize):
    # Calculate difference.
    df_error_XOSC32K = getVersavisError(file_XOSC32K, 'XOSC32K')
    df_error_DFLL48M = getVersavisError(file_DFLL48M, 'DFLL48M')
    df_error_EXT10MHZ = getVersavisError(file_EXT10MHZ, 'EXT10MHZ')
    df_error_ROSTIMENOW = getVersavisError(file_ROSTIMENOW, 'ROSTIMENOW')
    
    # Start data when ROSTIMENOW converged.
    df_error_XOSC32K = df_error_XOSC32K[df_error_XOSC32K['t'] > start]
    df_error_DFLL48M = df_error_DFLL48M[df_error_DFLL48M['t'] > start]
    df_error_EXT10MHZ = df_error_EXT10MHZ[df_error_EXT10MHZ['t'] > start]
    df_error_ROSTIMENOW = df_error_ROSTIMENOW[df_error_ROSTIMENOW['t'] > start]
    
    # Rename columns
    df_error_XOSC32K['clock'] = 'XOSC32K'
    df_error_DFLL48M['clock'] = 'DFLL48M'
    df_error_EXT10MHZ['clock'] = 'EXT10MHZ'
    df_error_ROSTIMENOW['clock'] = 'ROSTIMENOW'
    
    df_error_XOSC32K.rename(columns={'XOSC32K': 'error'}, inplace=True)
    df_error_DFLL48M.rename(columns={'DFLL48M': 'error'}, inplace=True)
    df_error_EXT10MHZ.rename(columns={'EXT10MHZ': 'error'}, inplace=True)
    df_error_ROSTIMENOW.rename(columns={'ROSTIMENOW': 'error'}, inplace=True)
    
    # Pick subset
    df_error_XOSC32K = df_error_XOSC32K[['error', 'clock']]
    df_error_DFLL48M = df_error_DFLL48M[['error', 'clock']]
    df_error_EXT10MHZ = df_error_EXT10MHZ[['error', 'clock']]
    df_error_ROSTIMENOW = df_error_ROSTIMENOW[['error', 'clock']]
    
    # Scale data.
    df_error_XOSC32K['error'] *= 10**6
    df_error_DFLL48M['error'] *= 10**6
    df_error_EXT10MHZ['error'] *= 10**6
    df_error_ROSTIMENOW['error'] *= 10**6
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    import itertools
    palette = itertools.cycle(sns.color_palette())
    fig, axes = plt.subplots(2, 2, figsize=(width,height))
    
    sns.violinplot(x='clock', y='error', data=df_error_XOSC32K, ax=axes[0, 0], color=next(palette), ylabel='blabla')
    sns.violinplot(x='clock', y='error', data=df_error_DFLL48M, ax=axes[0, 1], color=next(palette))
    sns.violinplot(x='clock', y='error', data=df_error_EXT10MHZ, ax=axes[1, 0], color=next(palette))
    sns.violinplot(x='clock', y='error', data=df_error_ROSTIMENOW, ax=axes[1, 1], color=next(palette))
    
    axes[0, 0].yaxis.grid(True)
    axes[0, 1].yaxis.grid(True)
    axes[1, 0].yaxis.grid(True)
    axes[1, 1].yaxis.grid(True)
    
    axes[0, 0].set_xlabel('')
    axes[0, 1].set_xlabel('')
    axes[1, 0].set_xlabel('')
    axes[1, 1].set_xlabel('')

    axes[0, 0].set_ylim(-160, 160)
    axes[0, 1].set_ylim(-160, 160)
    axes[1, 0].set_ylim(-1.6, 1.6)
    axes[1, 1].set_ylim(-4500, 4500)

    axes[0, 0].set_ylabel('')
    axes[0, 1].set_ylabel('')
    axes[1, 0].set_ylabel('')
    axes[1, 1].set_ylabel('')
    
    fig.text(0.04, 0.5, 'Timestamp error [us]', va='center', rotation='vertical')
    
    fig.savefig("plots/violin.pdf", bbox_inches='tight')

In [425]:
def plotClockEstimator(file, clock, clk_freq, width, height, fontsize):
    import rosbag_pandas
    import pandas as pd

    # Open file
    df = rosbag_pandas.bag_to_dataframe(file)

    # Get filter messages.
    df = df[['/versavis/gnss_sync/filter_state/x', '/versavis/gnss_sync/filter_state/z', '/versavis/gnss_sync/filter_state/P', '/versavis/gnss_sync/filter_state/Q', '/versavis/gnss_sync/filter_state/R']]
    df.dropna(inplace=True)

    # Rename states.
    state = 'Estimated bias ' + clock
    measurement = 'Measured PPS period ' + clock
    df.rename(columns={'/versavis/gnss_sync/filter_state/x':state}, inplace=True)
    df.rename(columns={'/versavis/gnss_sync/filter_state/z':measurement}, inplace=True)

    # Index to seconds since start
    df.index = pd.to_datetime(df.index, unit='s')
    df.reset_index(inplace=True)
    df.rename(columns={'index':'t'}, inplace=True)
    df['t'] = df['t'] - df['t'][0]
    df['t'] = df['t'].dt.round('1s') # Round to full seconds.
    df['t'] = df['t'].dt.total_seconds()

    # Plot state propagation
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(width,height))
    df.plot(ax=ax, x='t', y=state)
    ax.set_xlabel('Time since start [s]')
    ax.set_ylabel('Clock offset [ppm]')
    ax.ticklabel_format(axis='y', useOffset=False)

    # Plot standard deviation
    import numpy as np
    df['sigma'] = np.sqrt(df['/versavis/gnss_sync/filter_state/P'])
    import matplotlib.pyplot as plt
    plt.fill_between(df['t'].values, (df[state]-2*df['sigma']).values, (df[state]+2*df['sigma']).values, color='b', alpha=.15)

    # Plot measurements in ppm.
    df[measurement] = df[measurement] / clk_freq * 1.0e6
    df.plot(ax=ax, x='t', y=measurement, marker='o', linestyle="None")
    #ax2 = df.plot(ax=ax, secondary_y=True, x='t', y=measurement, marker='o', linestyle="None")
    #ax2.set_ylabel('PPS period [Ticks]')
    #ax2.ticklabel_format(axis='y', useOffset=False)
    #from matplotlib.ticker import MaxNLocator
    #ax2.yaxis.set_major_locator(MaxNLocator(integer=True))

    fig.savefig("plots/filter_" + clock + ".pdf", bbox_inches='tight')

In [ ]:
width = 12
height = 0.382 / 0.618 * width
fontsize = 12
start=120

file_XOSC32K = "XOSC32K_VERSAVIS_2020-01-27-18-21-53.bag"
file_DFLL48M = "DFLL48M_VERSAVIS_2020-01-27-18-17-56.bag"
file_EXT10MHZ = "EXT10MHZ_VERSAVIS_2020-01-27-18-13-58.bag"
file_ROSTIMENOW = "ROSTIMENOW_VERSAVIS_2020-01-27-18-46-45.bag"

plotTimeSeries(file_XOSC32K, file_DFLL48M, file_EXT10MHZ, file_ROSTIMENOW, start, width, height, fontsize)
#plotHistogram(file_XOSC32K, file_DFLL48M, file_EXT10MHZ, file_ROSTIMENOW, start, width, height, fontsize)
plotViolin(file_XOSC32K, file_DFLL48M, file_EXT10MHZ, file_ROSTIMENOW, start, width, height, fontsize)

plotClockEstimator(file_XOSC32K, 'XOSC32K', 32768, width, height, fontsize)
plotClockEstimator(file_DFLL48M, 'DFLL48M', 48e6, width, height, fontsize)
plotClockEstimator(file_EXT10MHZ, 'EXT10MHZ', 10e6, width, height, fontsize)